# **Práctica 4**: Regresión polinomial
## Especificaciones:
El *dataset* a utilizar en esta práctica es el de cal_housing.csv
- Las columnas *longitude, latitude, housingMedianAge, totalRooms, totalBedrooms, population, households, medianIncome* son las características de las casas.
- La columna *medianHouseValue* es el valor a predecir.

Con el dataset especificado realice lo siguiente:

- Cargue el dataset
- Genere un conjunto de entrenamiento con 80% de los datos y 20% para las pruebas
- Mezcle los datos con semilla 0
- Con el conjunto de entrenamiento, genere un conjunto de validación de 10 pliegues


In [41]:
import pandas as pd
import numpy as np
import operator
import os
from tabulate import tabulate


import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
class validation_set:
  def __init__(self, X_train, y_train, X_test, y_test):
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test

In [3]:
class test_set:
  def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test

In [4]:
class data_set:
  def __init__(self, validation_set, test_set):
    self.validation_set = validation_set
    self.test_set = test_set

In [5]:
def pliegues_validacion(pliegues, folder):
  # url = 'https://firebasestorage.googleapis.com/v0/b/portafoliowebsite.appspot.com/o/Notas%20Amatsu%2Fcal_housing.csv?alt=media&token=476a6d9d-ca90-42ba-9ab6-1dbfd9babdea'
  url = './dataset/cal_housing.csv'
  df = pd.read_csv(url, sep=',', engine='python')

  # corpus
  X = df[df.columns[0:8].values]
  # target
  y = df['medianHouseValue']
    
  #Separa corpus en conjunto de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state=0)

  validation_sets = []

  # Número de pliegues
  kf = KFold(n_splits=pliegues)
  c = 0

  for train_index, test_index in kf.split(X_train):
    c = c + 1
    i = 1
    X_train_v, X_test_v = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_v, y_test_v = y_train.iloc[train_index], y_train.iloc[test_index]
    
    #Agrega el pliegue creado a la lista
    validation_sets.append(validation_set(X_train_v, y_train_v, X_test_v, y_test_v))

    #Almacena el conjunto de prueba
    my_test_set = test_set(X_test, y_test)	
      
    #Guarda el dataset con los pliegues del conjunto de validación y el conjunto de pruebas
    my_data_set = data_set(validation_sets, my_test_set)

    my_data_set.test_set.X_test.to_csv(folder + 'data_test.csv', index = False)
    my_data_set.test_set.y_test.to_csv(folder + 'target_test.csv', index = False)

    cad_pliegues = str(pliegues) + '_'

    for val_set in my_data_set.validation_set:
      cad_i = str(i)
      val_set.X_train.to_csv(folder + 'data_validation_train_'+ cad_pliegues + cad_i + '.csv', index = False)
      val_set.y_train.to_csv(folder + 'target_validation_train_'+ cad_pliegues + cad_i+ '.csv', index = False)
      val_set.X_test.to_csv(folder + 'data_test_'+ cad_pliegues + cad_i+ '.csv', index = False)
      val_set.y_test.to_csv(folder + 'target_test_'+ cad_pliegues + cad_i + '.csv', index = False)
      i = i + 1
  print('Terminado los 10 pliegues')

In [6]:
k = 10
path = 'pliegues/'
# pliegues_validacion(k, path)

Usando el conjunto de entrenamiento realice lo siguiente con cada pliegue:

1. Utilice **regresión lineal mediante gradiente descendiente estocástico** para generar una línea que se ajuste lo mejor posible a los datos (modificando parámetros *max_iter, learning_rate y eta0*) y reporte el error cuadrado medio (MSE) y el coeficiente de determinación ($r^2$)


In [7]:
name_files = os.listdir('pliegues/')

In [8]:
data_train = list(filter(lambda x: x.startswith('data_validation_train'), name_files))
target_train = list(filter(lambda x: x.startswith('target_validation_train'), name_files))
print(len(data_train), len(target_train))

10 10


In [9]:
mse_list = []
r2_list = []

In [10]:
def sgdregressor (grade, x, y, learning, eta = 0.01, color = 'g', max = int(), i = int(), robusto = False, estandar = False):
  """
  grade : grado de la regresión polinomial
  x : datos de x
  y : datos de y
  learning : metodo de aprendizaje del SDGRegresor (constant, optimal, invscaling, adaptive)
  eta : valor de eta0 (defecto 0.01)
  max : número maximo de iteraciones
  color : posición de un vector con nombres de colores
  i : número de iteración
  robusto : valor boleano para ver si se escalan o no los datos
  estandar : valor booleano para ver si se escalan o no los datos
  """

  polynomial_features= PolynomialFeatures(degree=grade)
  x_poly = polynomial_features.fit_transform(x)
  escalado = 'No'

  if robusto:
    x_poly = preprocessing.RobustScaler().fit_transform(x_poly)
    escalado = 'Robusto'

    
  
  if estandar:
    x_poly = preprocessing.StandardScaler().fit_transform(x_poly)
    escalado = 'Estandar'
   
   
  regr = SGDRegressor(learning_rate = learning, eta0 = eta, max_iter= max)
  regr.fit(x_poly, y.values.ravel())
  y_poly_pred = regr.predict(x_poly)
  mse = mean_squared_error(y, y_poly_pred)
  r2 = r2_score(y, y_poly_pred)

  mse_list.append(mse)
  r2_list.append(r2)


  # ~ #Ajustes para que la curva trazada se vea correctamente
  sort_axis = operator.itemgetter(0)
  sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)

  # print (tuple(sorted_zip))
  x_sorted, y_poly_pred = zip(*sorted_zip)
  print(pd.DataFrame({'x': np.reshape(x_sorted,(1,-1))[0], 'Predicted': y_poly_pred}))

  # plt.plot(x_sorted, y_poly_pred, color='b')
  # plt.title('Gradiente estocástico, grado:'+ str(grade)+  ', escalado: '+ escalado)
  # plt.plot(x_sorted,y_poly_pred, color = color, label='pliegue'+str(i+1))
  
  # plt.scatter(x_sorted,y_poly_pred, color = color, label='pliegue'+str(i+1))

  # plt.legend()
  print ('Regresión polinomial estocástico grado {}\nmse: {} r2: {}'.format(grade, mse, r2))

In [11]:
def itera(grado, val_ap, val_e, val_itera, robusto = False, normal = False):

  # mse_list = []
  # r2_list = []
  colores = ['red', 'gray', 'green', 'black', 'orange', 'teal', 'yellow', 'coral', 'palegreen', 'aqua', 'cyan']

  for i, file in enumerate(data_train):

    print('Grado '+ str(grado) + ' k='+str(i))

    x = pd.read_csv(path + file)
    y = pd.read_csv(path + 'target_validation_train_10_'+ file[25:-4] + '.csv')

    # if tipo == 0:
    #   generate_polynomial(grado, x, y,'optimal',0.001, colores[i], i)
    # elif tipo == 1:

    sgdregressor(grado,x, y, val_ap, val_e, colores[i], val_itera, i, robusto= robusto, estandar = normal)

Después de definir las funciónes necesarias, vamos e empezar con el calculo de la regresión lineal de primer grado

2. Utilice **regresión polinomial mediante el gradiente descendiente estocástico** con poinomios de grado **2 y 3** para generar una línea curva que se ajuste lo mejor posible a los datos (modificando parámetros *max_iter, learning_rate y eta0*), y reporte MSE y $r^2$.


In [12]:
learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
grados = [1, 2, 3]
iter_max = 1000000000
eta = 0.000000001

for learning in learning_rate:
    for grado in grados:
        print(grado, learning)
        itera(grado, learning, eta, iter_max)
        # print(grado, learning)

1 constant
Grado 1 k=0
                  x      Predicted
0        households  283980.947623
1  housingMedianAge  225872.518273
2          latitude  231974.435408
3         longitude  222967.737905
4      medianIncome  318258.134876
5        population  211558.775685
6     totalBedrooms  229779.794074
7        totalRooms  181208.558810
Regresión polinomial estocástico grado 1
mse: 11413545886.231411 r2: 0.14364262772656533
Grado 1 k=1
                  x      Predicted
0        households  224376.376044
1  housingMedianAge  196875.536229
2          latitude  211176.089497
3         longitude  184599.386311
4      medianIncome  344844.915110
5        population  251309.781613
6     totalBedrooms  229261.086294
7        totalRooms  178560.497197
Regresión polinomial estocástico grado 1
mse: 11517745547.722565 r2: 0.14005270389980573
Grado 1 k=2
                  x      Predicted
0        households  223640.993893
1  housingMedianAge  194829.519176
2          latitude  208266.659447
3    

In [13]:
learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
grados = [1, 2, 3]
# escalados = [True, False]
iter_max = 1000000000
eta = 0.0001

for learning in learning_rate:
    for grado in grados:
        print(grado, learning)
        itera(grado, learning, eta, iter_max, normal=True)
        # print(grado, learning,escalado)

1 constant
Grado 1 k=0
                  x      Predicted
0        households  353886.814554
1  housingMedianAge  218127.900979
2          latitude  362631.758442
3         longitude  316017.854453
4      medianIncome  327822.335571
5        population  266260.522590
6     totalBedrooms  267220.524823
7        totalRooms   63221.283271
Regresión polinomial estocástico grado 1
mse: 4797291113.96371 r2: 0.640059657766783
Grado 1 k=1
                  x      Predicted
0        households  245997.332790
1  housingMedianAge  214349.389476
2          latitude  279674.910654
3         longitude  172522.717563
4      medianIncome  466480.580512
5        population  440906.588634
6     totalBedrooms  245567.649237
7        totalRooms  122194.986841
Regresión polinomial estocástico grado 1
mse: 4883957863.5586605 r2: 0.6353499613589728
Grado 1 k=2
                  x      Predicted
0        households  246559.774778
1  housingMedianAge  214838.324061
2          latitude  279908.284056
3         

In [14]:
learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
grados = [1, 2, 3]
# escalados = [True, False]
iter_max = 1000000000
eta = 0.0001

for learning in learning_rate:
    for grado in grados:
        print(grado, learning)
        itera(grado, learning, eta, iter_max, robusto=True)
        # print(grado, learning,escalado)

1 constant
Grado 1 k=0
                  x      Predicted
0        households  350086.913415
1  housingMedianAge  222430.828605
2          latitude  370149.227432
3         longitude  314145.356018
4      medianIncome  321367.859084
5        population  262081.007721
6     totalBedrooms  266407.646104
7        totalRooms   65541.925486
Regresión polinomial estocástico grado 1
mse: 4813627512.378784 r2: 0.6388339391900503
Grado 1 k=1
                  x      Predicted
0        households  247334.485347
1  housingMedianAge  214847.497584
2          latitude  279809.798540
3         longitude  175015.194300
4      medianIncome  462051.845870
5        population  443048.149361
6     totalBedrooms  245329.793297
7        totalRooms  122675.649886
Regresión polinomial estocástico grado 1
mse: 4882552918.393673 r2: 0.6354548585188202
Grado 1 k=2
                  x      Predicted
0        households  247322.598018
1  housingMedianAge  214255.033949
2          latitude  277838.927396
3        

3. Almacene los valores MSE y $r^2$ de cada pliegue y al terminar los 10 pliegues reporte el resultado de estos.


In [28]:
learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
# promedio_grado1_sin_escalado_constan = np.mean(np.array(mse_list[:9]))
# promedio_grado2_sin_escalado_constant= np.mean(np.array(mse_list[9:19]))
# promedio_grado3_sin_escalado_constant = np.mean(np.array(mse_list[19:28]))
# promedio_grado1_sin_escalado_optimal = np.mean(np.array(mse_list[28:]))

learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
grados = [1, 2, 3]
inicio = 0
final = 10
promedio_mse = []
for learning in learning_rate:
    print('antes ciclo:',inicio, final)
    for grado in grados:
        # print(learning, grado)
        # print(inicio, final)
        promedio_mse.append(np.mean(np.array(mse_list[inicio: final])))
        inicio +=10
        final += 10



antes ciclo: 0 10
antes ciclo: 30 40
antes ciclo: 60 70
antes ciclo: 90 100


In [30]:
learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
# promedio_grado1_sin_escalado_constan = np.mean(np.array(mse_list[:9]))
# promedio_grado2_sin_escalado_constant= np.mean(np.array(mse_list[9:19]))
# promedio_grado3_sin_escalado_constant = np.mean(np.array(mse_list[19:28]))
# promedio_grado1_sin_escalado_optimal = np.mean(np.array(mse_list[28:]))

learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
grados = [1, 2, 3]
inicio = 0
final = 10
promedio_r2 = []
for learning in learning_rate:
    print('antes ciclo:',inicio, final)
    for grado in grados:
        # print(learning, grado)
        # print(inicio, final)
        promedio_r2.append(np.mean(np.array(r2_list[inicio: final])))
        inicio +=10
        final += 10



antes ciclo: 0 10
antes ciclo: 30 40
antes ciclo: 60 70
antes ciclo: 90 100


In [31]:
len(promedio_r2)

12

In [39]:
# learning_rate =['constant', 'optimal', 'invscaling', 'adaptive']
# grados = [1, 2, 3]
# # escalados = [True, False]
# iter_max = 1000000000
# eta = 0.0001
def crea_tabla(vector_mse = list(), vector_r2 = list(), grado = int(), metodo = str(), iteraciones = int(), robusto = False, normal = True):
    """
    Creamos la tabla que se requiere para la entrega final.
    vector_mse : vector de promedios del error cuadratico
    vector_r2 : vector de promedios del r2
    grado : grado utilizado para el calculo
    metodo : lsita de la forma de aprender
    iteraciones : número de iteraciones que se uso para entrenar el modelo
    robusto : si se utilizo escalamiento o no
    normal : si se utilizo un escalado normal
    """
    escalado = 'no_escalado'

    if robusto:
        escalado = 'escalado_robusto'
    
    if normal:
        escalado = 'escalado_normal'
    
    if len(learning_rate) == len(vector_mse) and len(learning_rate) == len(vector_r2):
        vec = []
        # llave = str(grado)+ ' '+ str(escalado) + ' '+ str(val_ap)+ ' ' + str(val_itera)+ ' '+ 'MSE_promedio'+ ' '+ 'r2_promedio'+ 'pliegue_' + str(i)
        for i in range(len(learning_rate)):
            for grado in grados:
                # print(learning_rate[i], grado, promedio[i])
                # llave = str(grado)+ ' '+ str(escalado) + ' '+ str(learning_rate[i])+ ' ' + str(iter_max)+ ' '+ 'MSE_promedio'+ ' '+ 'r2_promedio
                # linea = str(grado) + ' '+ str(escalado) + ' ' + str(learning_rate[i]) + ' ' + str(iter_max) + ' ' + str(promedio_mse[i]) + ' ' + str(promedio_r2)
                vec.append([str(grado),str(escalado),str(metodo[i]),str(iteraciones),str(vector_mse[i]),str(vector_r2[i])])
                # vector.append(llave,)
        
        print(tabulate(vec,headers=['Tipo de regresion','Tipo de escalamiento', 'Learning rate utilizado', 'Número de iteraciones utilizadas', 'MSE promedio', 'r2 promedio'],tablefmt="grid", numalign="center"))

    else:
        print('Error en los datos.\n Revisalos')





+---------------------+------------------------+---------------------------+------------------------------------+----------------+---------------+
|  Tipo de regresion  | Tipo de escalamiento   | Learning rate utilizado   |  Número de iteraciones utilizadas  |  MSE promedio  |  r2 promedio  |
+=====================+========================+===========================+====================================+================+===============+
|          1          | no_escalado            | constant                  |             1000000000             |  1.15847e+10   |   0.134414    |
+---------------------+------------------------+---------------------------+------------------------------------+----------------+---------------+
|          2          | no_escalado            | constant                  |             1000000000             |  1.15847e+10   |   0.134414    |
+---------------------+------------------------+---------------------------+------------------------------------+-----